[Naive Bayes, Clearly Explained!!!](https://www.youtube.com/watch?v=O2L2Uv9pdDA)

In [32]:
import jlam.nbdev as nbdev
nbdev.parse_nb("Naive_Bayes.ipynb","naive_bayes.py")

In [30]:
#<<export>>
from typing import Set, NamedTuple
import re

In [3]:
def tokenize(text: str) -> Set[str]:
    text = text.lower() # Convert to lowercase
    all_words = re.findall("[a-z0-9]+",text) # extract the words
    return set(all_words) # remove duplicates

assert tokenize("Data Science is Science") == {"data",'science','is'}

In [4]:
#<<export>>
class Message(NamedTuple):
    text: str
    is_spam: bool

The constructor will take just one parameter, the pseudocount to use when computing probabilities. It also initializes an empty set of tokens, counters to track how often each token is seen in spam messages and ham messages, and counts of how many spam and ham messages it was trained on:


In [5]:
#<<export>>
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k # smoothing factor
        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0
        
    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Increment message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1
                
            # Increment word counts
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1
                    
    def _probabilities(self, token: str) -> Tuple[float, float]:
        """returns P(token | spam) and P(token | ham)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]
        
        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)
        
        return p_token_spam, p_token_ham
        
        
    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0
        
        #iterate through each word in our vocabulary
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)
            
            # If *token* appears in the message,
            # add the log probability of seeing it
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)
                
            # otherwise add the log probability of not seeing it
            # which is log(1 - probability of seeing it)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)
                
        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)
        
    

### Testing our model

In [6]:
messages = [Message("spam rules", is_spam=True),
           Message("ham rules", is_spam=False),
           Message("hello ham", is_spam=False)]

In [7]:
model = NaiveBayesClassifier(k=0.5)
model.train(messages)

### Check the counts

In [8]:
assert model.tokens == {"spam","ham","rules","hello"}

In [9]:
assert model.spam_messages == 1
assert model.ham_messages == 2

In [10]:
assert model.token_spam_counts == {"spam":1, "rules":1}
assert model.token_ham_counts == {"ham":2, "rules":1, "hello":1}

### Check by hand

In [11]:
text = "hello spam"

probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5), # "spam" _present
    1 - (0 + 0.5) / (1 + 2 * 0.5), # "ham" not present
    1 - (1 + 0.5) / (1 + 2 * 0.5), # "rules" not present
    (0 + 0.5) / (1 + 2 * 0.5) # "hello" present
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5), # "spam" _present
    1 - (2 + 0.5) / (2 + 2 * 0.5), # "ham" not present
    1 - (1 + 0.5) / (2 + 2 * 0.5), # "rules" not present
    (1 + 0.5) / (2 + 2 * 0.5), # "hello" present
]

In [12]:
p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

In [13]:
model.predict(text)

0.8350515463917525

In [14]:
p_if_spam / (p_if_spam + p_if_ham)

0.8350515463917525

In [15]:
assert model.predict(text) == p_if_spam / (p_if_spam + p_if_ham)

In [16]:

math.log(9,2)

3.1699250014423126

### Using Our Model

In [17]:
from io import BytesIO # So we can treat bytes as a file
import requests # To download the files, which
import tarfile  # are in .tar.bz format

In [18]:
BASE_URL = "https://spamassassin.apache.org/old/publiccorpus"
FILES = ["20021010_easy_ham.tar.bz2",
        "20021010_hard_ham.tar.bz2",
         "20021010_spam.tar.bz2"
        ]

In [19]:
OUTPUT_DIR = 'spam_data'
for filename in FILES:
    content = requests.get(f"{BASE_URL}/{filename}").content
    fin = BytesIO(content)
    with tarfile.open(fileobj=fin, mode='r:bz2') as tf:
        tf.extractall(OUTPUT_DIR)

In [20]:
!ls spam_data

easy_ham hard_ham spam


In [21]:
# look at subject line of each file
import glob, re

path = 'spam_data/*/*'

data: List[Message] = []
    
# glob.glob returns every filename that matches the wildcarded path
for filename in glob.glob(path):
    is_spam = "ham" not in filename
    
    # Thereare some garbage characters in the emails; skips them instead of raising an exception
    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break

### Splitting data into training and testing dataset

In [22]:
import random
from scratch.machine_learning import split_data

In [23]:
random.seed(0)
train_messages, test_messages = split_data(data, 0.75)

model = NaiveBayesClassifier()
model.train(train_messages)

In [24]:
from collections import Counter

predictions = [(message, model.predict(message.text))
                  for message in test_messages
              ]

# Assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)

confusion_matrix = Counter((message.is_spam, spam_probability > 0.5)
                            for message, spam_probability in predictions)



In [25]:
print("True Negative: (False, False) \n True Positive: (True, True) \n False Negative: (True, False) \n False Positive: (False, True)")
print(confusion_matrix)
    

True Negative: (False, False) 
 True Positive: (True, True) 
 False Negative: (True, False) 
 False Positive: (False, True)
Counter({(False, False): 664, (True, True): 85, (True, False): 54, (False, True): 22})


In [26]:
precision = 85 / (85 + 22)
precision

0.794392523364486

### inspect inner words

In [27]:
def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:
    # shouldn't call private method
    prob_if_spam, prob_if_ham = model._probabilities(token)
    return prob_if_spam / (prob_if_spam + prob_if_ham)


In [37]:
p_spam_dic = {}
for t in model.tokens:
    p_spam_dic[t]=p_spam_given_token(t,model)

In [39]:
import itertools

dict(itertools.islice(p_spam_dic.items(),5))

{'mirror': 0.6607310215557638,
 'bad': 0.1363724289122445,
 'affiliate': 0.8538554703270085,
 'yahoo': 0.2176597715344242,
 'copy': 0.7780502759043532}

In [43]:
words = sorted(model.tokens, key=lambda t: p_spam_given_token(t,model), reverse=True)
# words

In [49]:
cnt=0
max=10
for t in words:
    print(f'{t}: {p_spam_dic[t]}')
    cnt += 1
    if cnt > max:
        break

adv: 0.9945090782228829
rates: 0.9945090782228829
systemworks: 0.9919154923286508
money: 0.9919154923286508
zzzz: 0.9910720891804572
sale: 0.9900322163174271
attn: 0.9900322163174271
account: 0.9900322163174271
per: 0.9887181724686009
clearance: 0.9887181724686009
95: 0.9887181724686009


In [50]:
print("spammiest_words", words[-10:])
print("hammiest_words", words[:10])

spammiest_words ['selling', 'bliss', 'apt', 'ouch', 'sadev', 'zzzzteana', 'razor', 'users', '2', 'spambayes']
hammiest_words ['adv', 'rates', 'systemworks', 'money', 'zzzz', 'sale', 'attn', 'account', 'per', 'clearance']
